# Actividad Práctica Experimental 9

Esta actividad busca investigar acerca de la reducción de dimensionalidad y sus ventajas


## Importación de Librerías

In [ ]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Carga del Dataset


En este caso se usa el dataset iris

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

## Estandarización de Datos

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Reducción de Dimensionalidad usando PCA

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

### Varianza explicada para cada componente

In [ ]:
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print("\nVarianza explicada por cada componente:")
for i, (var, cum_var) in enumerate(zip(explained_variance, cumulative_variance)):
    print(f"Componente {i+1}: {var:.3f} ({cum_var:.3f} acumulada)")

### Reducción a 2 dimensiones

In [ ]:
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

### Visualización de la varianza y de las dimensiones reducidas

In [ ]:
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(
    range(1, len(explained_variance) + 1),
    cumulative_variance,
    marker='o'
)
plt.title('Varianza Explicada Acumulada')
plt.xlabel('Número de Componentes Principales')
plt.ylabel('Proporción de Varianza Explicada')
plt.grid()

plt.subplot(1, 2, 2)
for i, target_name in enumerate(target_names):
    plt.scatter(X_pca_2d[y == i, 0], X_pca_2d[y == i, 1], alpha=0.8, label=target_name)
plt.xlabel('Primer componente principal')
plt.ylabel('Segunda componente principal')
plt.title('PCA - Iris dataset')
plt.legend()
plt.tight_layout()
plt.show()

### Análisis

Los primeros dos componentes principales explican aproximadamente el 95.8% de la varianza (73.0% + 22.8%).

En el gráfico de dispersión, se observa una buena separación entre la clase setosa y las otras dos clases. Las clases versicolor y virginica muestran cierta superposición, pero aún se pueden distinguir en gran medida.

## Usando t-SNE

In [ ]:
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

## Comparación de ambos algoritmos

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
for i, target_name in enumerate(target_names):
    plt.scatter(X_pca_2d[y == i, 0], X_pca_2d[y == i, 1], alpha=0.8, label=target_name)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA')
plt.legend()

plt.subplot(1, 2, 2)
for i, target_name in enumerate(target_names):
    plt.scatter(X_tsne[y == i, 0], X_tsne[y == i, 1], alpha=0.8, label=target_name)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('t-SNE')
plt.legend()
plt.tight_layout()
plt.show()

### Comparación

1. t-SNE muestra una mejor separación entre las tres clases, especialmente entre versicolor y virginica que en PCA aparecían más superpuestas.

2. t-SNE parece más útil para visualización ya que maximiza la separación entre clases, aunque PCA es más interpretable ya que los componentes principales tienen significado en términos de las variables originales.

## Implementación de KNN

### Usando el conjunto de datos original

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
start_time = time.time()
knn_original = KNeighborsClassifier(n_neighbors=3)
knn_original.fit(X_train, y_train)
original_time = time.time() - start_time

y_pred_original = knn_original.predict(X_test)
accuracy_original = accuracy_score(y_test, y_pred_original)

### Usando los datos con PCA

In [ ]:
X_train_pca = pca_2d.transform(scaler.transform(X_train))
X_test_pca = pca_2d.transform(scaler.transform(X_test))

In [ ]:
start_time = time.time()
knn_pca = KNeighborsClassifier(n_neighbors=3)
knn_pca.fit(X_train_pca, y_train)
pca_time = time.time() - start_time

y_pred_pca = knn_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)

## Comparación de resultados

In [ ]:
print("\nComparación de resultados:")
print(f"Datos originales - Exactitud: {accuracy_original:.4f}, Tiempo entrenamiento: {original_time:.6f} seg")
print(f"PCA (2 componentes) - Exactitud: {accuracy_pca:.4f}, Tiempo entrenamiento: {pca_time:.6f} seg")

### Exactitud:

El modelo entrenado con el conjunto de los datos originales obtuvo un 100% de exactitud, mientras que el modelo en el que se usó PCA alcanzó un ~95%. Esto se debe a que el modelo entrenado con los datos originales dispone de todas las características, sin embargo, el modelo que usó PCA alcanzó una precisión bastante buena.

### Tiempo de entrenamiento:

El entrenamiento usando PCA es más rápido, pero en este dataset no es muy notoria la diferencia debido a que el dataset no cuenta con muchas características y cuenta con pocos datos.



## Preguntas de Control:


### 1. ¿Cómo cambia el rendimiento con reducción de dimensionalidad?


El rendimiento puede mejorar con la reducción de dimensionalidad cuando se elimina ruido o características irrelevantes, lo que ayuda a reducir el sobreajuste y acelera el entrenamiento del modelo. Sin embargo, si se pierde información relevante durante el proceso, el rendimiento puede disminuir, especialmente en conjuntos de datos donde todas las características son importantes para la predicción.



### 2. ¿En qué casos conviene aplicar PCA antes de entrenar un modelo?


Aplicar PCA antes de entrenar un modelo es conveniente cuando se trabaja con datos de alta dimensionalidad y multicolinealidad, o cuando se necesita reducir el costo computacional sin perder demasiada información. También es útil para visualizar datos en 2D o 3D, o cuando se busca eliminar ruido y simplificar la estructura de los datos antes de modelar.



### 3. ¿Qué ventajas tiene reducir dimensionalidad cuando se trata de modelos complejos o datos ruidosos?


Reducir la dimensionalidad en modelos complejos o datos ruidosos ofrece ventajas como una menor susceptibilidad al sobreajuste, ya que se eliminan características redundantes o irrelevantes. Además, mejora la eficiencia computacional al trabajar con menos variables y puede aumentar la interpretabilidad del modelo. En datos ruidosos, técnicas como PCA ayudan a extraer las componentes principales más significativas, descartando variabilidad no informativa.